# Yandex Algorithm 2018 ML Track

## простой baseline на основе Facebook fastText 
при подкрутке параметров может давать более 84000-85500

Будем использовать оригинальный python binding от Facebook:
  * основная страница https://fasttext.cc/docs/en/english-vectors.html
  * python binding собирао из trunk: https://github.com/facebookresearch/fastText/tree/master/python
  * обученную модель брал со следующей страницы: https://fasttext.cc/docs/en/pretrained-vectors.html

Весь workflow скопирван с предыдущего baseline от Cheremisin Oleg:
    https://github.com/applied-data-science/Data_Mining_in_Action_2018_Spring/blob/master/sport/hw2_yandex_algorithm_2018/benchmarks/Cheremisin_Oleg_easy_tfidf_baseline.ipynb

In [5]:
import fastText

In [7]:
ft_model = fastText.load_model("./fastText/cc.ru.300.bin")

In [9]:
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv('data/train.tsv', sep='\t', quoting=3, header=None, encoding="utf-8")

In [11]:
test = pd.read_csv('data/public.tsv', sep='\t', quoting=3, error_bad_lines=False, header=None, encoding="utf-8")

In [12]:
test.shape

(9968, 6)

In [13]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [14]:
train.sample(3)

,0,1,2,3,4,5,6,7
71270,205138670204625,"только не говори , что ты забыла , какой сегод...","ну , это не годовщина свадьбы , или мой день р...","поверить не могу , что ты забыла .",2,мне самой не верится .,good,0.927451
81962,236272223294384,вероятно .,"мы живем в мире , полном вероятностей .",ну . . .,0,"что , если это все ?",good,0.955260
43799,125709547554888,,я отдам своему отцу часть печени,у него печеночная недостаточность на последней...,5,""" и блестит в моих глазах """,bad,0.958676


In [15]:
test.sample(3)

,0,1,2,3,4,5
4753,136687627005334,он сейчас с иисусом .,а мама - нет .,"она с тобой , со мной и с джорджем .",0,другой со мной .
6014,170387729221106,,,"чувак , она оставила тебе деньги ?",0,да .
4629,133057534030884,моим родителям нравятся .,( смеясь ) :,"спасибо , что не говоришь "" предкам "" .",2,""" потому что дураки "" ."


In [16]:
# расставляем значения по формуле хорошесть * confidence
# Good (2)
# Neutral (1)
# Bad (0)

In [40]:
def rank2num(st):
    if st == 'good':
        return 2
    else:
        if st == 'neutral':
            return 1
        else:
            return 0

In [41]:
train['rank'] = train[6].apply(rank2num)
train['target'] = train['rank'] * train[7]
train.head()

,0,1,2,3,4,5,6,7,rank,target
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.875352,2,1.750703
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.900968,1,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.884320,0,0.000000
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.982530,2,1.965061
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.838054,2,1.676107


In [42]:
t1_ft = np.vstack(train[1].apply(lambda x: ft_model.get_sentence_vector(x)))
t2_ft = np.vstack(train[2].apply(lambda x: ft_model.get_sentence_vector(x)))
t3_ft = np.vstack(train[3].apply(lambda x: ft_model.get_sentence_vector(x)))
t5_ft = np.vstack(train[5].apply(lambda x: ft_model.get_sentence_vector(x)))

te1_ft = np.vstack(test[1].apply(lambda x: ft_model.get_sentence_vector(x)))
te2_ft = np.vstack(test[2].apply(lambda x: ft_model.get_sentence_vector(x)))
te3_ft = np.vstack(test[3].apply(lambda x: ft_model.get_sentence_vector(x)))
te5_ft = np.vstack(test[5].apply(lambda x: ft_model.get_sentence_vector(x)))

In [43]:
%%time

X_train = np.hstack([t1_ft, t2_ft, t3_ft, t5_ft])
X_test  = np.hstack([te1_ft, te2_ft, te3_ft, te5_ft])

CPU times: user 95.5 ms, sys: 80.1 ms, total: 176 ms
Wall time: 175 ms


In [44]:
X_train.shape, X_test.shape

((97533, 1200), (9968, 1200))

In [45]:
y_train = train['target']

In [46]:
train_part_size = int(0.75 * train['target'].shape[0])
X_train_part = X_train[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid =  X_train[train_part_size:, :]
y_valid = y_train[train_part_size:]

In [47]:
X_train_part.shape

(73149, 1200)

In [25]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

reg = Ridge()

reg.fit(X_train_part, y_train_part)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [26]:
reg_pred = reg.predict(X_valid)

In [27]:
# Oleg говорит, что метрика ниже - неправильная, но на практике её снижение даёт уверенный рост на лидерборде

In [28]:
%%time
from sklearn.metrics import mean_absolute_error

valid_mae = mean_absolute_error(y_valid, reg_pred)
print(valid_mae)

0.7190280100517679
CPU times: user 2.21 ms, sys: 1.22 ms, total: 3.44 ms
Wall time: 19.1 ms


In [29]:
%%time
reg.fit(X_train, y_train)

CPU times: user 2.94 s, sys: 342 ms, total: 3.28 s
Wall time: 572 ms


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [30]:
y_test = reg.predict(X_test)

In [31]:
sub = pd.DataFrame()

In [32]:
test.sample(3)

,0,1,2,3,4,5
3720,105136523138492,"и я очень рад , что он наконец нашел кого - то","мы должны радоваться за него , а не . . вмешив...","нет , я не забочусь о его личной жизни",5,заботиться о нем ?
5460,154513595864713,,властям нужны костюмы .,"как железный человек , понимаешь ?",2,чего не понимаю ?
4773,137137147493903,,"да , была бы .",но я не стал ее делать .,2,не делаешь чего ?


In [33]:
sub['context_id'] = test[0]
sub['reply_id'] = test[4]
sub['rank'] = - y_test

In [34]:
sub.head()

,context_id,reply_id,rank
0,138920940977,0,-1.034704
1,138920940977,1,-1.335494
2,138920940977,2,-1.060796
3,138920940977,3,-1.462555
4,138920940977,4,-1.494842


In [35]:
submission = sub.sort_values(by=['context_id', 'rank'])

In [36]:
del submission['rank']

In [37]:
submission.head()

,context_id,reply_id
4,138920940977,4
3,138920940977,3
1,138920940977,1
5,138920940977,5
2,138920940977,2


In [38]:
test.shape, sub.shape

((9968, 6), (9968, 3))

In [39]:
submission.to_csv('yandex-ml-naive-fasttext.tsv',header=None, index=False, sep=' ')